In [1]:
import sys; sys.path.append('..'); from wwa import *

## Check available models
```
synda search variable=tos,ta,hus,psl -l 10000 > ~/wwa/24-08+typhoon-gaemi/cmip6-pi-vars.txt
```

In [45]:
df = pd.read_table("cmip6-pi-vars.txt", sep = "\.", engine = "python", header = None,
                  names = [0,"activity","institute","source_id","experiment","variant_label","table", "variable", "grid", "version"])

In [46]:
# identify models with all required variables available in both historical & SSP experiments
nvars = 5

mdl_xvar = df[["source_id", "variant_label", "variable", "experiment"]].drop_duplicates()
mdl_xvar = mdl_xvar.groupby(["source_id", "variant_label"]).count().reset_index()

mdl_xvar = mdl_xvar.loc[mdl_xvar.variable == (2*nvars)]

In [47]:
mdl_xvar

,source_id,variant_label,variable,experiment
8,CESM2,r10i1p1f1,10,10
9,CESM2,r11i1p1f1,10,10
13,CESM2,r4i1p1f1,10,10
23,CESM2-WACCM,r1i1p1f1,10,10
24,CESM2-WACCM,r2i1p1f1,10,10
...,...,...,...,...
385,MRI-ESM2-0,r5i1p1f1,10,10
390,NESM3,r1i1p1f1,10,10
391,NESM3,r2i1p1f1,10,10
395,NorESM2-LM,r1i1p1f1,10,10


In [48]:
# filter models to show only those with all four variables
df = pd.merge(df, mdl_xvar, left_on = ["source_id", "variant_label"], right_on = ["source_id", "variant_label"], how = "inner")

# get list of model variants to include in synda download
print("c+p into synda defaults file:")
"source_id="+",".join(sorted(df.source_id.drop_duplicates().to_list()))

c+p into synda defaults file:


'source_id=CESM2,CESM2-WACCM,CMCC-CM2-SR5,CMCC-ESM2,CNRM-CM6-1,CNRM-CM6-1-HR,CNRM-ESM2-1,CanESM5,EC-Earth3,EC-Earth3-CC,EC-Earth3-Veg,EC-Earth3-Veg-LR,IPSL-CM6A-LR,MIROC6,MPI-ESM1-2-HR,MPI-ESM1-2-LR,MRI-ESM2-0,NESM3,NorESM2-LM,NorESM2-MM'

In [49]:
len(df.source_id.drop_duplicates())

20

In [50]:
# which variant labels provide most runs?
nmod = df.groupby(["variant_label"]).count()[["source_id"]] / (nvars*2)
nmod.sort_values("source_id")

,source_id
variant_label,
r6i1p1f2,1.3
r9i1p2f1,1.8
r2i1p2f1,1.8
r3i1p2f1,1.8
r5i1p2f1,1.8
...,...
r6i1p1f1,7.2
r4i1p1f1,10.4
r3i1p1f1,13.2


In [25]:
# list models with all available data for run r1i1p1f1 (this is the single most commonly used label)
df.loc[df.variant_label == "r1i1p1f1", ["source_id", "grid"]].drop_duplicates().sort_values("source_id")

,source_id,grid
218,AWI-CM-1-1-MR,gn
106,BCC-CSM2-MR,gn
93,CESM2-WACCM,gn
92,CESM2-WACCM,gr
296,CMCC-CM2-SR5,gn
298,CMCC-ESM2,gn
102,CanESM5,gn
66,EC-Earth3,gn
62,EC-Earth3-CC,gn
58,EC-Earth3-Veg,gn


In [12]:
# count of model variants
df[["source_id", "variant_label"]].drop_duplicates().groupby(["source_id"]).count()

,variant_label
source_id,
CMCC-ESM2,1
CanESM5,50
EC-Earth3,58
EC-Earth3-Veg,1
FGOALS-g3,3
INM-CM4-8,1
INM-CM5-0,1
IPSL-CM6A-LR,7
KACE-1-0-G,3
